### Gerador de amostras aleatórias para treinamento do time de processamento de dados (inventário florestal) da OPPY Ambiental

#### Amostra de mudas recém-plantadas

#### Parâmetros:
- Núcleação
- 22 mudas por núcleo
- Proporção ecológica 12-8-2
- Criação de condição fitossanitária
- Estimativas dendrométricas compatíveis com plantio de até 1 ano


#### Agosto de 2021

In [2]:
import pandas as pd
import numpy as np

# The file seems to use ';' as a delimiter, let's reload it with the correct delimiter.
species_data = pd.read_csv('/home/oppy/Dell/Downloads/WAE/Reposicao_2023/MUDAS/lista_lean_prompt.csv', delimiter=';')

# Check the first few rows again to ensure it's loaded correctly
print(species_data.head())


# Set the parameters
num_samples = 15
group_size = 22
total_records = num_samples * group_size
status_options = ["Sadia", "Morta-geada", "Morta-seca", "Morta-predação-tatu", "Morta-predação-lagarta"]
dac_mean, dac_std = 0.89, 0.2  # DAC in cm
height_mean, height_std = 61.3, 15  # Height in cm

# Prepare lists to store data
ids = []
amostras = []
nome_popular = []
nome_cientifico = []
ecologia = []
dac_values = []
height_values = []
statuses = []

# Filter species by ecological type
climax_species = species_data[species_data['Ecologia'].str.contains('Climácica')]
secondary_species = species_data[species_data['Ecologia'].str.contains('Secundária')]
pioneer_species = species_data[species_data['Ecologia'].str.contains('Pioneira')]

# Function to assign status to each group
def assign_statuses(num_sadias):
    other_statuses_count = group_size - num_sadias
    return ["Sadia"] * num_sadias + np.random.choice(status_options[1:], other_statuses_count).tolist()

# Loop through the number of samples (amostras)
for amostra in range(1, num_samples + 1):
    # Sample species from each group
    climax_sample = climax_species.sample(2, replace=True)
    secondary_sample = secondary_species.sample(8, replace=True)
    pioneer_sample = pioneer_species.sample(12, replace=True)
    
    # Combine samples
    sample_species = pd.concat([climax_sample, secondary_sample, pioneer_sample])
    
    # Generate random DAC and height values for each plant
    dac_sample = np.random.normal(dac_mean, dac_std, group_size).round(2)
    height_sample = np.random.normal(height_mean, height_std, group_size).round(2)
    
    # Randomly assign status, with 17-22 being "Sadia"
    num_sadias = np.random.randint(17, 23)
    sample_statuses = assign_statuses(num_sadias)
    
    # Append data to lists
    for i in range(group_size):
        ids.append(len(ids) + 1)
        amostras.append(amostra)
        nome_popular.append(sample_species.iloc[i]['Nome popular'])
        nome_cientifico.append(sample_species.iloc[i]['Nome científico'])
        ecologia.append(sample_species.iloc[i]['Ecologia'])
        dac_values.append(dac_sample[i])
        height_values.append(height_sample[i])
        statuses.append(sample_statuses[i])

# Create the final DataFrame
sampling_data = pd.DataFrame({
    'id': ids,
    'Amostra': amostras,
    'Nome científico': nome_cientifico,
    'Nome popular': nome_popular,
    'Ecologia': ecologia,
    'Diâmetro à altura do colo (DAC) (cm)': dac_values,
    'Altura (cm)': height_values,
    'Status': statuses
})

      id  Quantidade de mudas plantadas       Nome popular  \
0  643.0                             20  Pitanga/Cerejeira   
1  663.0                            200          Guabiroba   
2  723.0                            131             Tarumã   
3  648.0                             20         Corticeira   
4  642.0                            131              Cedro   

            Nome científico    Ecologia  
0       Eugenia involucrata  Secundária  
1  Campomanesia xanthocarpa  Secundária  
2        Vitex megapotamica   Climácica  
3           Annona rugulosa  Secundária  
4         Cedrella fissilis   Climácica  


In [3]:
# Show the first few rows of the generated data
sampling_data.head()

,id,Amostra,Nome científico,Nome popular,Ecologia,Diâmetro à altura do colo (DAC) (cm),Altura (cm),Status
0,1,1,Matayba elaeagnoides,Camboatá-branco,Climácica,1.00,68.78,Sadia
1,2,1,Cedrella fissilis,Cedro,Climácica,1.05,52.07,Sadia
2,3,1,Gochnatia polymorpha,Cambará,Secundária,0.58,73.43,Sadia
3,4,1,Tabebuia alba,Ipê-amarelo,Secundária,1.11,68.49,Sadia
4,5,1,Cordia trichotoma,Louro-pardo,Secundária,1.03,35.96,Sadia


In [4]:
# Saving as a .csv file for other uses (Excel, WPS Office, R...)
sampling_data.to_csv('amostragem_teste.csv', index=False)

#### Fim